In [4]:
import cv2
import numpy as np
import math
import scipy.linalg

In [5]:
img_dir = '/home/rushi/College/Fall_2022/Perception/HW1/Q2/IMG_7745.png'
image = cv2.imread(img_dir)
# print(np.shape(image))
gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur_img = cv2.GaussianBlur(gray_img, (5,5), 1)
edge_img = cv2.Canny(blur_img, 150, 200, None, 3)  ## to find edges

lines = cv2.HoughLinesP(edge_img, 1, np.pi/180, 200, maxLineGap=3000)  # to find end point coordinates of lines
x_list = [7,19,246,304,6,151]
for i in x_list:
    # if i == 28:
        x1, y1, x2, y2 = lines[i][0]
        cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 3)
        cv2.putText(image, str(i), (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 3)

# cv2.namedWindow("image", cv2.WINDOW_NORMAL)    
# cv2.imshow("image", image)


cv2.waitKey()

-1

In [6]:
line_list = [7,19,246,304,6,151]


m = np.zeros(6)
b = np.zeros(6)

for i in range(len(line_list)):
    x1, y1, x2, y2 = lines[line_list[i]][0]
    m[i] = (y2 - y1)/(x2 - x1)
    b[i] = y1 - m[i]*x1

    if line_list[i] == 7 or line_list[i] == 19:
        cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 3)

    elif line_list[i] == 246 or line_list[i] == 304:
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 3)

    else:
        cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 3)

cv2.imwrite('/home/rushi/College/Fall_2022/Perception/HW1/Q2/vanishing_lines.png', image)

l1 = np.array([m[0], -1, b[0]])
l2 = np.array([m[1], -1, b[1]])
l3 = np.array([m[2], -1, b[2]])
l4 = np.array([m[3], -1, b[3]])
l5 = np.array([m[4], -1, b[4]])
l6 = np.array([m[5], -1, b[5]])

inter_point = np.cross(l1, l2)
inter_point2 = np.cross(l3, l4)
inter_point3 = np.cross(l5, l6)
c = np.eye(3)*3100
x_1 = int(inter_point[0]/inter_point[2])
y_1 = int(inter_point[1]/inter_point[2])
x_2 = int(inter_point2[0]/inter_point2[2])
y_2 = int(inter_point2[1]/inter_point2[2])
x_3 = int(inter_point3[0]/inter_point3[2])
y_3 = int(inter_point3[1]/inter_point3[2])

# print(x_1, y_1)
# print(x_2, y_2)
# print(x_3, y_3)

In [7]:
# calibrating camera
vanishing_poiints = [[x_1, y_1], [x_2 ,y_2], [x_3,y_3]]
A = []

for i, p_i in enumerate(vanishing_poiints):
    for j , p_j in enumerate(vanishing_poiints):
        if i!=j and j>0:
            p_i_homo = [p_i[0],p_i[1],1.0]
            p_j_homo = [p_j[0],p_j[1],1.0]
            A.append([p_i_homo[0]*p_j_homo[0]+p_i_homo[1]*p_j_homo[1],p_i_homo[0]*p_j_homo[2]+p_i_homo[2]*p_j_homo[0], p_i_homo[1]*p_j_homo[2]+p_i_homo[2]*p_j_homo[1],
                          p_i_homo[2]*p_j_homo[2]])

A = np.array(A, dtype=np.float32)
A = A[:-1]

u, s, v_t = np.linalg.svd(A, full_matrices=True)

m1, m4, m5, m6 = v_t.T[:,-1]

m = np.array([[m1, 0., m4],
                [0., m1, m5],
                [m4, m5, m6]])


m = m + np.eye(3)
K_transpose_inv = np.linalg.cholesky(m)
K = np.linalg.inv(K_transpose_inv.T)

K = K / K[-1, -1]
K = K + c
K[2][2] = K[2][2]/3100

print(K)

focal_len = np.hypot(K[0][0], K[1][1])
print(focal_len)
cv2.namedWindow("image", cv2.WINDOW_NORMAL)
cv2.imshow("image", image)

cv2.waitKey()


[[3.10141421e+03 0.00000000e+00 1.33874003e-04]
 [0.00000000e+00 3.10141421e+03 1.12138245e-04]
 [0.00000000e+00 0.00000000e+00 1.00032258e+00]]
4386.062043282853
